In [1]:
import pandas as pd
import os
from sodapy import Socrata
from sqlalchemy import create_engine
import psycopg2 as pg
import pandas.io.sql as psql


# # get connected to the database
# connection = pg.connect("dbname=postgres user=root password=Murphy8034?")
 
# dataframe = psql.frame_query("SELECT * FROM data_df", connection)


In [2]:
# 1. Create a DataFrame Covid data(full csv). 
data_df = pd.read_csv('/Users/benjaminshelburn/Desktop/Classwork/Final Project/COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv')

# 2. Create a DF from small API sample - limit 250K?
# client = Socrata("data.cdc.gov", None)
# data = client.get("n8mc-b4w4", limit= 500000)
# data_df = pd.DataFrame.from_records(data)
# data_df

# # 3. https://stackoverflow.com/questions/66345379/not-getting-full-data-from-api-to-jupyter-notebook
# r = requests.get("https://pokeapi.co/api/v2/pokemon/")
# x = r.json()

# data = []
# data.append(x['results'])

# # Loop until `next` is `None`
# while x.get('next'):
#     r = requests.get("https://pokeapi.co/api/v2/pokemon/")
#     x = r.json()
#     data.append(x['results'])

# df = pd.DataFrame(data)

/Users/benjaminshelburn/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# Columns to drop
data_df = data_df.drop(['state_fips_code'], axis=1)
data_df = data_df.drop(['county_fips_code'], axis=1)
data_df = data_df.drop(['case_positive_specimen_interval'], axis=1)
data_df = data_df.drop(['current_status'], axis=1)
data_df = data_df.drop(['case_onset_interval'], axis=1)
data_df = data_df.drop(['ethnicity'], axis=1)
data_df = data_df.drop(['process'], axis=1)
data_df = data_df.drop(['res_county'], axis=1)


# Print df
data_df

,case_month,res_state,age_group,sex,race,exposure_yn,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
0,2020-12,AR,0 - 17 years,NaN,NaN,Missing,Symptomatic,Missing,Missing,NaN,NaN
1,2021-10,MO,0 - 17 years,NaN,NaN,Missing,Missing,Unknown,Missing,Unknown,NaN
2,2020-10,MO,0 - 17 years,Female,NaN,Missing,Symptomatic,Unknown,Missing,Unknown,NaN
3,2021-08,TN,0 - 17 years,Female,Missing,Missing,Missing,Missing,Missing,Missing,NaN
4,2020-11,GA,0 - 17 years,Female,Unknown,Missing,Symptomatic,Missing,Missing,Missing,NaN
...,...,...,...,...,...,...,...,...,...,...,...
37532067,2020-11,SD,65+ years,Male,White,Missing,Symptomatic,Missing,Missing,Missing,NaN
37532068,2020-11,SD,65+ years,Male,White,Missing,Symptomatic,Missing,Missing,Missing,NaN
37532069,2020-11,SD,65+ years,Male,White,Missing,Symptomatic,Missing,Missing,Missing,NaN
37532070,2020-11,SD,65+ years,Male,White,Missing,Symptomatic,Missing,Missing,Missing,NaN


In [4]:
# Change and format column names
data_df.rename(columns={'case_month':'Case Month', 'res_state':'Resident State', 'res_county': 'Resident County',
                       'age_group':'Age Group', 'sex':'Sex', 'race':'Race', 'ethnicity': 'Ethnicity',
                       'process':'Process', 'exposure_yn': 'Exposure', 'symptom_status': 'Symptom Status',
                       'hosp_yn':'Hospitalization', 'icu_yn': 'ICU', 'death_yn':'Death',
                        'underlying_conditions_yn': 'Underlying Health Conditions'},inplace=True )

In [5]:
# Copy df and insert blank Final Outcome column
data_df = data_df.copy()
data_df['Final Outcome']=""
data_df

,Case Month,Resident State,Age Group,Sex,Race,Exposure,Symptom Status,Hospitalization,ICU,Death,Underlying Health Conditions,Final Outcome
0,2020-12,AR,0 - 17 years,NaN,NaN,Missing,Symptomatic,Missing,Missing,NaN,NaN,
1,2021-10,MO,0 - 17 years,NaN,NaN,Missing,Missing,Unknown,Missing,Unknown,NaN,
2,2020-10,MO,0 - 17 years,Female,NaN,Missing,Symptomatic,Unknown,Missing,Unknown,NaN,
3,2021-08,TN,0 - 17 years,Female,Missing,Missing,Missing,Missing,Missing,Missing,NaN,
4,2020-11,GA,0 - 17 years,Female,Unknown,Missing,Symptomatic,Missing,Missing,Missing,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...
37532067,2020-11,SD,65+ years,Male,White,Missing,Symptomatic,Missing,Missing,Missing,NaN,
37532068,2020-11,SD,65+ years,Male,White,Missing,Symptomatic,Missing,Missing,Missing,NaN,
37532069,2020-11,SD,65+ years,Male,White,Missing,Symptomatic,Missing,Missing,Missing,NaN,
37532070,2020-11,SD,65+ years,Male,White,Missing,Symptomatic,Missing,Missing,Missing,NaN,


In [6]:
# remove rows using the drop() function
data_df.drop(data_df.index[data_df['Resident State'] == 'Missing'], inplace=True)
#data_df.drop(data_df.index[data_df['Resident County'] == 'Missing'], inplace=True)
#data_df.drop(data_df.index[data_df['Age Group'] == 'Missing'], inplace=True)
#data_df.drop(data_df.index[data_df['Age Group'] == 'NA'], inplace=True)
#data_df.drop(data_df.index[data_df['Sex'] == 'Missing'], inplace=True)
#data_df.drop(data_df.index[data_df['Sex'] == 'NA'], inplace=True)
#data_df.drop(data_df.index[data_df['Sex'] == 'Unknown'], inplace=True)
#data_df.drop(data_df.index[data_df['Race'] == 'Missing'], inplace=True)
#data_df.drop(data_df.index[data_df['Race'] == 'NA'], inplace=True)
#data_df.drop(data_df.index[data_df['Race'] == 'Unknown'], inplace=True)
#data_df.drop(data_df.index[data_df['Symptom Status'] == 'Unknown'], inplace=True)
#data_df.drop(data_df.index[data_df['Symptom Status'] == 'Missing'], inplace=True)

#data_df.drop(data_df.index[data_df['Ethnicity'] == 'Missing'], inplace=True)

# display the dataframe
data_df

,Case Month,Resident State,Age Group,Sex,Race,Exposure,Symptom Status,Hospitalization,ICU,Death,Underlying Health Conditions,Final Outcome
0,2020-12,AR,0 - 17 years,NaN,NaN,Missing,Symptomatic,Missing,Missing,NaN,NaN,
1,2021-10,MO,0 - 17 years,NaN,NaN,Missing,Missing,Unknown,Missing,Unknown,NaN,
2,2020-10,MO,0 - 17 years,Female,NaN,Missing,Symptomatic,Unknown,Missing,Unknown,NaN,
3,2021-08,TN,0 - 17 years,Female,Missing,Missing,Missing,Missing,Missing,Missing,NaN,
4,2020-11,GA,0 - 17 years,Female,Unknown,Missing,Symptomatic,Missing,Missing,Missing,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...
37532067,2020-11,SD,65+ years,Male,White,Missing,Symptomatic,Missing,Missing,Missing,NaN,
37532068,2020-11,SD,65+ years,Male,White,Missing,Symptomatic,Missing,Missing,Missing,NaN,
37532069,2020-11,SD,65+ years,Male,White,Missing,Symptomatic,Missing,Missing,Missing,NaN,
37532070,2020-11,SD,65+ years,Male,White,Missing,Symptomatic,Missing,Missing,Missing,NaN,


In [7]:
data_df.to_csv(r'/Users/benjaminshelburn/Desktop/Classwork/Final Project/CleanData.csv', index = False)


In [8]:
preferred_cities_df.count()


Case Month                      37532060
Resident State                  37531152
Age Group                       37125438
Sex                             36287796
Race                            30934656
Exposure                        37532072
Symptom Status                  37532072
Hospitalization                 37532072
ICU                             37532072
Death                           35779158
Underlying Health Conditions     2325769
Final Outcome                   37532072
dtype: int64